# Goal
To generate polymers from the bottom up where the generation of monomers is implicitly guided to resemble a dataset of small molecules and where the polymer assembly is guided towards specified desired properties. Both the monomer and polymer level objectives are encouraged to increase diversity of the assembled chemical structure through a diversity object and also to obey the underlying chemical rules through a validity objective.

This algorithm is designed as a reinforcement learning agent (RL agent) that operates within a chemistry aware graph generation environment. A molecule is successively constructed by either connecting a new substructure or an atom with an existing molecular graph or adding a bond to connect existing atoms. GCPN predicts the action of the bond addition, and is trained via policy gradient to optimize a reward composed of molecular property objectives and adversarial loss. The adversarial loss is provided by a graph convolutional network based discriminator trained jointly on a dataset of example molecules. Overall, this approach allows direct optimization of application-specific objectives, while ensuring that the generated molecules are realistic and satisfy chemical rules.

# Problem Solution Details
* Graph G is represented as: $G \in (A, E, F)$
* Adjacency Matrix A is represented as (assuming n nodes):  $A \in\{0,1\}^{n \times n}$
* Node Feature Matrix F (assuming each node has d features): $F \in \mathbb{R}^{n \times d}$
* (Discrete) Edge-Conditioned Adjacency tensor (assuming b possible edge types): $E \in\{0,1\}^{b \times n \times n}$
    * $E_{i, j, k}=1$ if there exists an edge of type i between nodes j and k
    * and $A=\sum_{i=1}^b E_i$
* Our primary objective is to generate graphs that maximize a given property function: $S(G) \in \mathbb{R}$ aka maximize 
    * $\mathbb{E}_{G^{\prime}}\left[S\left(G^{\prime}\right)\right]$ where $G^{\prime}$ is the generated graph and $S$ could be one or multiple domain-specific statistics of interest.
* Constrain our model with two main sources of prior knowledge:
    * (1) Generated graphs need to satisfy a set of hard constraints
        * Chemical Valency
    * (2) We provide the model with a set of example graphs $G \sim p_{\text {data }}(G)$ and would like to incorporate such prior knowledge by regularizing the property optimization objective with $\mathbb{E}_{G, G^{\prime}}\left[J\left(G, G^{\prime}\right)\right]$ under distance metric $J(\cdot, \cdot)$
        * this distance metric is an adversarially trained discriminator


# Molecule Generation Environment
The environment builds up a molecular graph step by step through a sequence of bond or substructure addition actions given by our agent.

## State Space
The state of the environment at timestep $t$ is $s_t$  and represents the intermediate generated graph $G_t$. This is fully observable by the agent. $G_0$ can be seen as the start of the generation process and starts as a single node that represents a carbon atom.

## Action Space
Special note: we must have a fixed-dimension and homogeneous action space to be amenable to the reinforcement learning framework.

### Link Prediciton Action
* set of scaffold subgraphs to be added during graph generation: $\left\{C_1, \ldots, C_s\right\}$ which can be defined as $C=\bigcup_{i=1}^s C_i$
* Given a $G_t$ at step $t$ we define the corresponding extended graph as $G_t \cup C$
* Therefore an action can either correspond to connecting a new subgraph $C_i$ to a node $G_t$ or connecting existing nodes within the graph $G_t$
* Once an action is taken, the remaining disconnected scaffold subgraphs are removed

To start we will use the most fine-grained version of $C$ which consists of all $b$ different single node graphs, where $b$ denotes the number of different atom types (you can modify this without affecting the integrity of the algorithm). Also note that $C$ can be extended to contain molecule substructure scaffolds this will allow for specification of preferred substructures but at the cost of model flexibility.

## Environment Dynamics
Domain-specific rules are incorporated in the state transition dynamics. The environment carries out actions that obey the given rules. Infeasible actions proposed by the policy network are rejected and the state remains unchanged. For the task of molecule generation, the environment incorporates rules of chemistry.  All actions that attempt to update the environment according to the environment dynamics $p\left(G_{t+1} \mid G_t, a_t\right)$ must (a) pass the valency check on the partial molecule (intermediate graph) according to the actions. Completed graphs must pass the valency check and also a (b) chemical validity check.

Note the function below `check_valency` is used in the step-wise step-wise valency check conducted on the partial molecule (intermediate graph). Chemical validity checks on a final graph use the function `check_chemical_validity`.


### Validation pieces of code
```python
from rdkit import Chem

def check_chemical_validity(self, mol: Chem.rdchem.Mol):
    """
    Checks the chemical validity of the mol object. Existing mol object is
    not modified. Radicals pass this test.
    :return: True if chemically valid, False otherwise
    """
    s = Chem.MolToSmiles(mol, isomericSmiles=True)
    m = Chem.MolFromSmiles(s)  # implicitly performs sanitization
    if m:
        return True
    else:
        return False

def check_valency(self, mol: Chem.rdchem.Mol):
    """
    Checks that no atoms in the mol have exceeded their possible
    valency
    :return: True if no valency issues, False otherwise
    """
    try:
        Chem.SanitizeMol(mol,
                         sanitizeOps=Chem.SanitizeFlags.SANITIZE_PROPERTIES)
        return True
    except ValueError:
        return False
```

## Reward Design
Intermediate rewards and final rewards are used to guide the behaviour of the agent.
* Final rewards as a sum over domain-specific rewards and adversarial rewards
    * The domain-specific rewards consist of the (combination of) final property scores
        * Domain-specific rewards also include penalization of unrealistic molecules according to various criteria, such as excessive steric strain and the presence of functional groups that violate ZINC functional group filters
        * Note that the Domain-specific rewards are provided through a surrogate model trained according to [alghani 2022](https://arxiv.org/abs/2205.08619) using the [MPNN 2017](https://arxiv.org/abs/1704.01212) architecture implemented in [dgl-lifesci](https://github.com/awslabs/dgl-lifesci). For now this is fine as we want something non-experimental to train and get working; however, this will change when we consider 3D generation in November 2022. Perhaps we can even leverage pretrained 2D models in when considering 3D via methods like [Stärk 3DInfoMax 2022](https://github.com/HannesStark/3DInfomax).
        
* Intermediate rewards include step-wise validity rewards and adversarial rewards
    * A small positive reward is assigned if the action does not violate valency rules, otherwise a small negative reward is assigned

When the environment updates according to a terminating action (selected by the agent), both a step reward and a final reward are given, and the generation process terminates.


### Reward Scale
When summing up all the rewards collected from a molecule generation trajectory, the range of the reward value that the model can get is $[-4,4]$, for final chemical property reward, $[-2,2]$ for final chemical filter reward, $[-1,1]$ for final adversarial reward, $[-1,1]$ for intermediate adversarial reward and $[-1,1]$ for intermediate validity reward.

The philosophy behind the reward design, we linearly scale each reward component according to its importance in molecule generation from a chemistry point of view as well as the quality of generation results.

# Constrained Optimization
## Adversarial Training
TL;DR Goal of Adversarial Training is to incorporate prior knowledge specified by a dataset of example molecules.

It is possible to hand code the rules or even to train a predictor for one of the properties; however, precisely representing the combination of these properties is extremely challenging. Adversarial training addresses the challenge through a learnable discriminator adversarially trained with a generator. After the training converges, the discriminator implicitly incorporates the information of a given dataset and guides the training of the generator.

### Equation 1-A
$$
\min _\theta \max _\phi V\left(\pi_\theta, D_\phi\right)=\mathbb{E}_{x \sim p_{\text {data }}}\left[\log D_\phi(x)\right]+\mathbb{E}_{x \sim \pi_\theta}\left[\log D_\phi(1-x)\right]
$$
* $-V\left(\pi_\theta, D_\phi\right)$ adversarial rewards from the GAN framework
* $\pi_\theta$ is the policy network
* $D_\phi$ is the discriminator network
* $x$ represents the input graph
* $p_{\text { data}}$ is the underlying data distribution which is defined either over final graphs (for final rewards) or intermediate graphs (for intermediate rewards)

The discriminator network employs the same architecture as the policy network used to calculate node embeddings which are then aggregated into a graph level embedding and used to make a property prediction.

Note this comes from the Minimax Loss function introduced in the [original GAN paper](https://arxiv.org/pdf/1406.2661.pdf): 
$\min _G \max _D V(D, G)=\mathbb{E}_{\boldsymbol{x} \sim p_{\text {data}}(\boldsymbol{x})}[\log D(\boldsymbol{x})]+\mathbb{E}_{\boldsymbol{z} \sim p_{\boldsymbol{z}}(\boldsymbol{z})}[\log (1-D(G(\boldsymbol{z})))]$

The generator tries to minimize the following function while the discriminator tries to maximize it.
* $D(x)$ is the discriminator's estimate of the probability that real data instance x is real.
* $E_x$ is the expected value over all real data instances.
* $G(z)$ is the generator's output when given noise z.
* $D(G(z))$ is the discriminator's estimate of the probability that a fake instance is real.
* $E_z$ is the expected value over all random inputs to the generator (in effect, the expected value over all generated fake instances $G(z)$).

The generator can't directly affect the $\log D(\boldsymbol{x})$ term in the function, so, for the generator, minimizing the loss is equivalent to minimizing $\log (1-D(G(\boldsymbol{z})))$.

Note that if we stay with this style of constrained optimization we will likely move to a more modern GAN loss function like [EMD](https://en.wikipedia.org/wiki/Earth_mover's_distance)

## MolDQN Similarity Score

Similarity is defined as the Tanimoto similarity between Morgan fingerprints with radius 2 of the generated molecule $m$ and the original molecule $m_0$: $\operatorname{SIM}\left(m, m_0\right) \geq \delta$ for a threshold $\delta$

[MolDQN](https://www.nature.com/articles/s41598-019-47148-x) trained a model in an environment whose initial state was randomly set to be one of the 800 molecules on the ZINC dataset which have the lowest penalized logP value, and ran the trained model on each molecule for one episode. The maximum number of steps per episode was limited to 20 in consideration of computational efficiency. In this task, the reward was designed as follows: 

$$\mathcal{R}(s)= \begin{cases}\log \mathrm{P}(m)-\lambda \times\left(\delta-\operatorname{SIM}\left(m, m_0\right)\right) & \text { if } \operatorname{SIM}\left(m, m_0\right)<\delta \\ \operatorname{logP}(m) & \text { otherwise }\end{cases}$$ where $\lambda$ is the coefficient to balance the similarity and logP. If the similarity constraint is not satisfied, the reward is penalized by the difference between the target and current similarity. $\lambda$ of 100 is used. 

### Equation 1-B

Tanimoto similarity: $T(a, b)=\frac{N_c}{N_a+N_b-N_c}$, $N_a$ and $N_b$ represents the number of attributes in each object (a, b) and $N_c$ is the number of attributes in common


## Suspected Affect on Diversity of Discovered Chemical Structure
We believe that constrained optimization has an affect on the diversity of discovered chemical structures.

# Graph Representation
The graph generation environment uses a policy network learned by the RL agent to act in the environment and can take an intermediate graph $G_t$ and the collection of scaffold subgraphs $C$ as inputs, and outputs the action $a_t$, which predicts a new link to be added.

## Computing Node Embeddings
We first compute node embeddings of an input graph before we can perform link prediction $G_t \cup C$. A GNN variant is used to support categorical edge types.

The high-level idea is to perform message passing over each edge type for a total of $L$ layers. At the $l^{th}$ layer of the GNN all messages from different edge types are aggregated and used to compute the next layer node embedding $H^{(l+1)} \in \mathbb{R}^{(n+c) \times k}$, where $n$ and $c$ are the sizes of $G_t$ and $C$ and $k$ (64 to start) is the embedding dimension.

### Equation 2
$$
H^{(l+1)}=\operatorname{AGG}\left(\operatorname{ReLU}\left(\left\{\tilde{D}_i^{-\frac{1}{2}} \tilde{E}_i \tilde{D}_i^{-\frac{1}{2}} H^{(l)} W_i^{(l)}\right\}, \forall i \in(1, \ldots, b)\right)\right)
$$

* $E_i$ is the $i^{th}$ slice of the edge-conditioned adjacency tensor $E$
* $\tilde{E}_i=E_i+I$$\tilde{D}_i=\sum_k \tilde{E}_{i j k}$ where $\tilde{D}$ is a learnable degree matrix
* $W_i^{(l)}$ is a trainable weight matrix for the $i^{th}$ edge type; using 4 to start (no bond, single bond, double bond, * bond)
* $H^{(l)}$ is the node representation learned in the $l^{th}$ layer; we're using $L$ == 3 to start
* $\operatorname{AGG}(\cdot)$ means some form of aggregation $\{$ MEAN, MAX, SUM, CONCAT $\}$...we'll try a few
* The $L$ layer GNN is applied to the extended graph $G_t \cup C$ to compute the final node embedding matrix $X=H^{(L)}$

## Action Prediction

The link prediction based action at $a_t$ time step $t$ is a concatenation of four components: selection of two nodes, prediction of edge type, and prediction of termination

Concretely, each component is sampled according to a predicted distribution governed by equations 3 and 4.


### Equation 3
$$
a_t=\operatorname{CONCAT}\left(a_{\text {first }}, a_{\text {second }}, a_{\text {edge }}, a_{\text {stop }}\right)
$$

### Equation 4
$$
\begin{array}{ll}
f_{\text {first }}\left(s_t\right) \operatorname{SOFTMAX}\left(m_f(X)\right), & a_{\text {first }} \sim f_{\text {first }}\left(s_t\right) \in\{0,1\}^n \\
f_{\text {second }}\left(s_t\right)=\operatorname{SOFTMAX}\left(m_s\left(X_{a_{\text {first }}}, X\right)\right), & a_{\text {second }} \sim f_{\text {second }}\left(s_t\right) \in\{0,1\}^{n+c} \\
f_{\text {edge }}\left(s_t\right)=\operatorname{SOFTMAX}\left(m_e\left(X_{a_{\text {first }}}, X_{a_{\text {second }}}\right)\right), & a_{\text {edge }} \sim f_{\text {edge }}\left(s_t\right) \in\{0,1\}^b \\
f_{\text {stop }}\left(s_t\right)=\operatorname{SOFTMAX}\left(m_t(\mathrm{AGG}(X)),\right. & a_{\text {stop }} \sim f_{\text {stop }}\left(s_t\right) \in\{0,1\}
\end{array}
$$

#### MLP 1
$m_f$ to denote a MLP that maps $Z_{0: n} \in \mathbb{R}^{n \times k}$ to $\mathbb{R}^n$ vector which represents the probability of selecting each node. Information from the first selected
node $a_{\text {first }}$ is incorporated in the selection of the second node by concatenating its embedding $Z_{a_{\text {first }}}$ with that of each node in $G_t \cup C$

#### MLP 2
The second MLP $ms$ then maps the concatenated embedding to the probability distribution of each potential node to be selected as the second node. Note that when selecting two nodes to predict a link, the first node to select, $a_{\text {first }}$, should always belong to the currently generated graph $G_t$, whereas the second node to select, $a_{\text {second }}$, can be either from $G_t$ (forming a cycle), or from $C$ (adding a new substructure)

#### MLP 3
To predict a link $m_e$ takes $Z_{a_{\text {first }}}$ and $Z_{a_{\text {second }}}$ as inputs and maps to a categorical edge type using an MLP

#### MLP 4
The termination probability is computed by firstly aggregating the node embeddings into a graph embedding using an aggregation function $\operatorname{AGG}$ then mapping the graph embedding to a scalar using an MLP $m_t$

# Reinforcement Learning
* Reinforcement learning is capable of directly representing hard constraints and desired properties through the design of environment dynamics and reward function.
* Reinforcement learning allows active exploration of the molecule space beyond samples in a dataset.

## Policy Gradient Training
Policy gradient based methods are widely adopted for optimizing policy networks. To start we're going to use Proximal Policy Optimization (PPO) for ease of implementation...lots of room for improvement here. Below equation 5 is the PPO objective function

## Equation 5
$$
\max L^{\text {CLIP }}(\theta)=\mathbb{E}_t\left[\min \left(r_t(\theta) \hat{A}_t, \operatorname{clip}\left(r_t(\theta), 1-\epsilon, 1+\epsilon\right) \hat{A}_t\right)\right], r_t(\theta)=\frac{\pi_\theta\left(a_t \mid s_t\right)}{\pi_{\theta_{o l d}}\left(a_t \mid s_t\right)}
$$
* $r_t(\theta)$ is the probability ratio that is clipped to the range of $[1-\epsilon, 1+\epsilon]$
* $L^{\operatorname{CLIP}}(\theta)$ is a lower bound of the conservative policy iteration objective
* $\hat{A}_t$ is the estimated advantage function which involves a learned value function $V_\omega(\cdot)$ to reduce the variance of estimation; note that $V_\omega(\cdot)$ is an MLP that maps the graph embedding computed according described in the section above

### Imitation Learning

Any ground truth molecule could be viewed as an expert trajectory for pretraining our agent. This expert imitation objective can be written as: $\min L^{\mathrm{EXPERT}}(\theta)=-\log \left(\pi_\theta\left(a_t \mid s_t\right)\right)$ where $\left(s_t, a_t\right)$ pairs are obtained from ground truth molecules.

Note: It is known that pretraining a policy network with expert policies if they are available leads to better training stability and performance...this is a very common first step in most rl training regimes.

Given a molecule dataset, we randomly sample a molecular graph $G$, and randomly select one connected subgraph $G^{\prime}$ of $G$ as the state $s_t$. At $s_t$ any action that adds an atom or bond in $G \backslash G^{\prime}$ can be taken in order to generate the sample molecule. Meaning randomly sample $a_t \in G \backslash G^{\prime}$ and use $\left(s_t, a_t\right)$ pairs to supervise the expert imitation objective.

#### Discuss the affect of expert imitation on molecular diversity.